# Chain of Thought
Imagine you have a task for which a model struggles to find the correct answer because it jumps to conclusions rather than thinking step-by-step

`Q: Sofia has 7 apples in her basket. Her friend Emily gives her 3 more bags of apples. Each bag contains 4 apples. How many apples does Sofia have in total?`
`step 1: calculate number of apples in the 3 bags from Emily`
`step 2: calculate total number of apples`
`step 3: add results of step1 and step1`
`A: 19 apples`
`Q: At the bakery, there are 12 cupcakes on a tray. Sarah takes 3 cupcakes for herself. hEr friend Alex then takes half of the remaining cupcakes. How many cupcakes are left on the tray?`
`A: ??`

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(usecwd=True))

True

In [3]:
messages = [(
    "user", """
Q: Sofia has 7 apples in her basket. Her friend Emily gives her 3 more bags of apples. Each bag contains 4 apples. How many apples does Sofia have in total?
step 1: calculate number of apples in the 3 bags from Emily
step 2: calculate total number of apples
step 3: add results of step1 and step1
A: 19 apples
Q: At the bakery, there are 12 cupcakes on a tray. Sarah takes 3 cupcakes for herself. hEr friend Alex then takes half of the remaining cupcakes. How many cupcakes are left on the tray?
A: ??
"""
)]

In [4]:
prompr_template = ChatPromptTemplate.from_messages(messages)
MODEL_NAME = 'openai/gpt-oss-120b'
model = ChatGroq(model_name=MODEL_NAME)

chain = prompr_template | model


In [6]:
response = chain.invoke({})

In [9]:
from pprint import pprint 
pprint(response.model_dump()["content"])

('**Step‑by‑step solution**\n'
 '\n'
 '1. **Start with the original number of cupcakes**  \n'
 '   - There are **12 cupcakes** on the tray.\n'
 '\n'
 '2. **Sarah takes 3 cupcakes**  \n'
 '   - Remaining cupcakes = 12\u202f−\u202f3 = **9 cupcakes**.\n'
 '\n'
 '3. **Alex takes half of the remaining cupcakes**  \n'
 '   - Half of 9 = 9\u202f÷\u202f2 = **4.5 cupcakes**.  \n'
 '   - Alex takes 4.5 cupcakes.\n'
 '\n'
 '4. **Find how many cupcakes are left on the tray**  \n'
 '   - Remaining cupcakes = 9\u202f−\u202f4.5 = **4.5 cupcakes**.\n'
 '\n'
 '---\n'
 '\n'
 '**Answer:** There are **4.5 cupcakes** left on the tray. (If only whole '
 'cupcakes can be taken, the problem would need whole‑number amounts; with the '
 'numbers given, the exact mathematical result is 4.5.)')


## Zero-Shot Chain-of-Thought (CoT)
 Zero-shot CoT reasoning leverages the model's ability to break down a complex problem into smaller, manageable steps. With zero-shot prompting, you can achieve results even without prior examples.

 By appending the instruction `Please think step-by-step` to the prompt, the model is encouraged to articulate its reasoning process clearly and sequentially. This technique aids in solving problems by guiding the model to consider each part of the question methodically, enhancing its capacity to reach a correct answer through logical deduction.

 Simply append the instruction `Please think step-by-step` to the prompt to help solve the problem

## Coding: Self-Consistency Chain-of-Thought
Ensuring self-consistency involves running CoT model several times with the same prompt. Ideally, different paths are sampled. All results are analysed, and the popular answer is selected

-Taks: `Four numbers are defined. Find the right arithmetic operations(+, -, *, /), that combines the numbers so that the result is 24`

In [24]:
from pprint import pprint
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(usecwd=True))

True

In [38]:
# function for zero-shot Chain-of-Thought Prompting
def chain_of_thought_prompting(prompt: str, model_name: str = 'openai/gpt-oss-120b') -> str:
    model = ChatGroq(model_name=model_name)
    prompt = ChatPromptTemplate.from_messages(messages=[
        ("system", "You're a helpful assistant and answer precise and concise."),
        ("user", f"{prompt} \n. Please think step by step")
    ])
    print(prompt)
    chain = prompt | model
    return chain.invoke({}).content

In [48]:
#  self_consisitency CoT
def self_consistency_cot(prompt: str, number_of_runs: int = 3) -> str:
    res = []
    for _ in range(number_of_runs):
        current_res = chain_of_thought_prompting(prompt)
        print(current_res)
        res.append(current_res)
    res_concat = ";".join(res)
    self_consistency_prompt = f"You will get multiple answers in <<>>, separated by ; <<{res_concat}>> . Extract only the final equations and return the most common equation as it was provided originally. If there is no common equation, return the most likely equation."
    # self_consistency_prompt_concat = ";".join(self_consistency_prompt)
    # messages = [
    #     ("system", "You are a helpful assistant and answer precise and concise."),
    #     ("user", f"{self_consistency_prompt_concat}")
    # ]
    safe_text = self_consistency_prompt.replace("{", "{{").replace("}", "}}")
    prompt = ChatPromptTemplate.from_messages([
       ("system", "You are a helpful assistant and answer precise and concise."),
        ("user", safe_text)
    ])
    # prompt = ChatPromptTemplate.from_messages(messages=messages)
    model = ChatGroq(model_name='openai/gpt-oss-120b')
    chain = prompt | model
    return chain.invoke({}).content

In [39]:
 #%% Test
user_prompt = """The goal of the Game of 24 is to use the four arithmetic operations (addition, subtraction, multiplication, and division) to combine four numbers and 
get a result of 24. The numbers are 3, 4, 6, and 8. It is mandatory to use all four numbers. Please check the final equation for correctness. Hints: Identify the basic operations,Prioritize multiplication and division, Look for combinations that make 
numbers divisible by 24, Consider order of operations, Use parentheses 
strategically, Practice with different number combinations"""
 
 # %% Single CoT
chain_of_thought_prompting(prompt=user_prompt)

input_variables=[] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template="You're a helpful assistant and answer precise and concise."), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='The goal of the Game of 24 is to use the four arithmetic operations (addition, subtraction, multiplication, and division) to combine four numbers and \nget a result of 24. The numbers are 3, 4, 6, and 8. It is mandatory to use all four numbers. Please check the final equation for correctness. Hints: Identify the basic operations,Prioritize multiplication and division, Look for combinations that make \nnumbers divisible by 24, Consider order of operations, Use parentheses \nstrategically, Practice with different number combinations \n. Please think step by step'), additional_kwargs={})]


'**Solution**\n\nUse the four numbers \\(3,\\;4,\\;6,\\;8\\) each exactly once and combine them with the basic arithmetic operations.\n\n\\[\n\\boxed{(8+4)\\times\\frac{6}{3}=24}\n\\]\n\n**Step‑by‑step check**\n\n1. **Parentheses first**  \n   \\[\n   8+4 = 12\n   \\]\n\n2. **Division**  \n   \\[\n   \\frac{6}{3}=2\n   \\]\n\n3. **Multiplication** (the final operation)  \n   \\[\n   12 \\times 2 = 24\n   \\]\n\nAll four numbers are used, only the allowed operations (+,\u202f/,\u202f×) are employed, and the result is exactly 24. Hence the equation is correct.'

In [47]:
res = self_consistency_cot(prompt=user_prompt, number_of_runs=3)
res

input_variables=[] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template="You're a helpful assistant and answer precise and concise."), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='The goal of the Game of 24 is to use the four arithmetic operations (addition, subtraction, multiplication, and division) to combine four numbers and \nget a result of 24. The numbers are 3, 4, 6, and 8. It is mandatory to use all four numbers. Please check the final equation for correctness. Hints: Identify the basic operations,Prioritize multiplication and division, Look for combinations that make \nnumbers divisible by 24, Consider order of operations, Use parentheses \nstrategically, Practice with different number combinations \n. Please think step by step'), additional_kwargs={})]
**Solution**

\[
(8-6)

ValueError: unexpected '{' in field name